In [ ]:
# Minimal CNN using ResNet (ResNet50) for image classification (Colab-ready)
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, resnet50

# --- load small subset of CIFAR-10 to avoid OOM ---
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, y_train = x_train[:4000], y_train[:4000]   # reduced training set
x_test,  y_test  = x_test[:800],  y_test[:800]      # reduced validation set

# --- one-hot encode labels (10 classes) ---
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test  = tf.keras.utils.to_categorical(y_test, 10)

# --- dataset pipeline: resize per-sample and apply ResNet preprocessing ---
def preprocess(img, label):
    img = tf.image.resize(img, (96, 96))            # smaller size -> less memory
    img = tf.cast(img, tf.float32)
    img = resnet50.preprocess_input(img)            # model-specific scaling
    return img, label

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
            .shuffle(1000).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(16).prefetch(tf.data.AUTOTUNE))

val_ds = (tf.data.Dataset.from_tensor_slices((x_test, y_test))
          .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
          .batch(16).prefetch(tf.data.AUTOTUNE))

# --- build model: ResNet50 backbone (frozen) + small classifier head ---
tf.keras.backend.clear_session()
base = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# --- compile, train, evaluate ---
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, epochs=3, validation_data=val_ds)
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc:.4f}")
